# **Projeto - Redes Convolucionais**

### **Download dataset**

In [33]:
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.python.keras.callbacks import EarlyStopping

In [34]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2022-05-16 02:24:20--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz.3’

images.tar.gz.3     100%[===================>] 755.23M  21.6MB/s    in 56s     

2022-05-16 02:25:16 (13.5 MB/s) - ‘images.tar.gz.3’ saved [791918971/791918971]

--2022-05-16 02:25:16--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/x-gzip]
Saving to: ‘annotations.tar.gz.3’

annotations.tar.gz. 100%[===================>]  18.28M  11.7MB/s    in 1.6s    

2022-05-16 02:25:18 (

In [35]:
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

### **Prepare data** 

In [36]:
IMAGE_BASE_PATH = 'images'
SAMPLE_FILES = 'annotations/list.txt'
TRAIN_FILES = 'annotations/trainval.txt'
TEST_FILES = 'annotations/test.txt'

files = open(TRAIN_FILES)
train_files = []
train_labels = []
for txt_line in files:
    if txt_line[0] == '#':
        continue
    file_name, class_id, cat_dog_id, sub_class_id = txt_line.split(' ')
    train_files.append(file_name + '.jpg')
    train_labels.append(class_id)
files.close()

train_data_frame = pd.DataFrame({'filename': train_files, 'label': train_labels})

In [37]:
BATCH_SIZE = 64

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale=1/255)

train_dataset = train_datagen.flow_from_dataframe(dataframe=train_data_frame,
                                        directory=IMAGE_BASE_PATH,
                                        x_col='filename',
                                        y_col='label',
                                        target_size=(128, 128),
                                        class_mode='categorical',
                                        batch_size=BATCH_SIZE,
                                        subset='training',
                                        shuffle=True,
                                        seed=42)

val_dataset = train_datagen.flow_from_dataframe(dataframe=train_data_frame,
                                      directory=IMAGE_BASE_PATH,
                                      x_col='filename',
                                      y_col='label',
                                      target_size=(128, 128),
                                      class_mode='categorical',
                                      batch_size=BATCH_SIZE,
                                      subset='validation',
                                      shuffle=True,
                                      seed=42)

Found 2944 validated image filenames belonging to 37 classes.
Found 736 validated image filenames belonging to 37 classes.


In [38]:
files = open(TEST_FILES)
test_files = []
test_labels = []
for txt_line in files:
    if txt_line[0] == '#':
        continue
    file_name, class_id, cat_dog_id, sub_class_id = txt_line.split(' ')
    test_files.append(file_name + '.jpg')
    test_labels.append(class_id)
files.close()

test_data_frame = pd.DataFrame({'filename': test_files, 'label': test_labels})

In [39]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

test_dataset = test_datagen.flow_from_dataframe(dataframe=test_data_frame,
                                        directory=IMAGE_BASE_PATH,
                                        x_col='filename',
                                        y_col='label',
                                        target_size=(128, 128),
                                        class_mode='categorical',
                                        batch_size=1,
                                        shuffle=False,
                                        seed=42)

Found 3669 validated image filenames belonging to 37 classes.


### **Model - ResNet50 Transfer Learning TF hub**

In [40]:
import tensorflow_hub as hub

In [41]:
resnet50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'

In [46]:
pretrained_base = hub.KerasLayer(resnet50, trainable = False) 

model = Sequential([
    pretrained_base,
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.3),
    Dense(64,activation='relu'),
    Dropout(0.3),
    Dense(64,activation='relu'),
    Dense(37, activation='softmax')])

In [47]:
model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['accuracy'],
    )

es = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 10, verbose = 1, mode = 'auto')


hist = model.fit_generator(train_dataset, steps_per_epoch = train_dataset.samples // 64, 
                           epochs = 30, validation_data = val_dataset, 
                           validation_steps = val_dataset.samples // 64, callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
46/46 [==============================] - 19s 354ms/step - loss: 3.8127 - accuracy: 0.1590 - val_loss: 3.2848 - val_accuracy: 0.1094
Epoch 2/30
46/46 [==============================] - 15s 342ms/step - loss: 2.3788 - accuracy: 0.3580 - val_loss: 2.7505 - val_accuracy: 0.1960
Epoch 3/30
46/46 [==============================] - 16s 347ms/step - loss: 1.9179 - accuracy: 0.4817 - val_loss: 2.1777 - val_accuracy: 0.3338
Epoch 4/30
46/46 [==============================] - 15s 340ms/step - loss: 1.6389 - accuracy: 0.5326 - val_loss: 1.9391 - val_accuracy: 0.4048
Epoch 5/30
46/46 [==============================] - 16s 344ms/step - loss: 1.4406 - accuracy: 0.5880 - val_loss: 1.7976 - val_accuracy: 0.4190
Epoch 6/30
46/46 [==============================] - 15s 339ms/step - loss: 1.2705 - accuracy: 0.6270 - val_loss: 1.6917 - val_accuracy: 0.4688
Epoch 7/30
46/46 [==============================] - 15s 342ms/step - loss: 1.1653 - accuracy: 0.6566 - val_loss: 1.7148 - val_accuracy: 0.4560

**Observações:**

Como foram atingidos valores baixos de acurácia tanto para a rede CNN, quanto para as redes propostas na literatura para a solução identificadora de raças, foi implementado também um terceiro modelo para essa solução utilizando o TensorFlow Hub. Observamos que valores maiores de acurácia são obtidos com esse tipo de modelo, porém ocorreu overfitting mesmo com Dropout.